In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
import numpy as np
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Define date intervals and corresponding CSV file paths
date_intervals = {
    '1-4July': [182, 183, 184, 185],
    '5-8July': [186, 187, 188, 189],
    '9-12July': [190, 191, 192, 193],
    '13-16July': [194, 195, 196, 197],
    '17-20July': [198, 199, 200, 201],
    '21-24July': [202, 203, 204, 205],
    '25-28July': [206, 207, 208, 209],
    '29-31July': [210, 211, 212]
}

# Base path to the CSV files (adjust if needed)
base_csv_path = r'Path_Sensor_Dataset'
# Initialize list to store all results
final_results = []

In [ ]:
for interval, days_to_analyze in date_intervals.items():
    # Construct the file path for the CSV
    csv_file_path = f'{base_csv_path} {interval}.csv'
    
    # Load the data for the current interval
    df = pd.read_csv(csv_file_path, low_memory=False)
    
    # Extract unique sensors
    sensor_values = df['Sensor'].unique()

    # Initialize list to store results for this interval
    results = []

    # Loop through each sensor
    for sensor in sensor_values:
        data1 = df[df['Sensor'] == sensor]
        data = data1.drop(['Device', 'Parent Device', 'Sensor', 'Location', 'Status', 'Units'], axis=1)
        data[['Date', 'Time']] = data['Time'].str.extract(r'(.+?\d{4}) (.+)')
        
        # Convert 'Time' and 'Date' to datetime with specified format
        data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d %b, %Y %I:%M:%S %p')
        
        # Convert Time to total seconds since midnight and Date to day of the year
        data['TotalSeconds'] = data['DateTime'].dt.hour * 3600 + data['DateTime'].dt.minute * 60 + data['DateTime'].dt.second
        data['DayOfYear'] = data['DateTime'].dt.dayofyear

        # Convert 'Value' to numeric (handling commas)
        data['Value'] = data['Value'].str.replace(',', '')
        data['Value'] = pd.to_numeric(data['Value'], errors='coerce', downcast='integer')
        data = data.dropna(subset=['Value'])

        # Loop through each day in the interval
        for day in days_to_analyze:
            df_day = data[data['DayOfYear'] == day]

            # Select features and target variable
            X = df_day[['TotalSeconds', 'DayOfYear']]
            y = df_day['Value']

            # Check if data is available for the day
            if len(X) == 0:
                print(f"No data available for {sensor} on day {day}")
                continue

            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

            # Initialize the Random Forest Regressor
            model = RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42)

            # Train the model
            model.fit(X_train, y_train)

            # Predict on the test set
            y_pred = model.predict(X_test)

            # Calculate metrics
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            # Store the results
            results.append({
                'Sensor': sensor,
                'DayOfYear': day,
                'MAE': mae,
                'MSE': mse,
                'RMSE': rmse,
                'R-squared': r2
            })

    # Create a DataFrame from results and save to Excel file for the current interval
    results_df = pd.DataFrame(results)
    excel_file_path = f'New folder/R7/R7_{interval}.xlsx'
    results_df.to_excel(excel_file_path, index=False)

    # Display the results DataFrame
    print(f"Results for {interval}:")
    print(results_df)

    # Store the interval results in the final results list
    final_results.append(results_df)

    # Provide download link for the Excel file
    print(f"Results saved to {excel_file_path}")


In [ ]:
# Optionally, combine all results into one DataFrame
combined_results_df = pd.concat(final_results, ignore_index=True)

# Save the combined results to an Excel file
combined_excel_file_path = 'New folder/R7/R7_July_Combined.xlsx'
combined_results_df.to_excel(combined_excel_file_path, index=False)

print("Combined results for all intervals saved to:", combined_excel_file_path)